<h1 style="text-align: center;">User Expenses Prediction - Data Preparation</h1>

In [39]:
import pandas as pd 
import numpy as np

In [16]:
# Sheet has two tabs so we import both tabs
df1 = pd.read_excel('dataset.xlsx', sheet_name=0)  # First sheet
df2 = pd.read_excel('dataset.xlsx', sheet_name=1)  # Second sheet

In [37]:
mode_df1 = df1['Customer ID'].value_counts().to_dict()
top_5_df1 = dict(list(mode_df1.items())[:5])
print(top_5_df1)

{14911.0: 5710, 17841.0: 5114, 14606.0: 3927, 14156.0: 2710, 12748.0: 2665}


In [38]:
mode_df2 = df2['Customer ID'].value_counts().to_dict()
top_5_df2 = dict(list(mode_df2.items())[:5])
print(top_5_df2)

{17841.0: 7983, 14911.0: 5903, 14096.0: 5128, 12748.0: 4642, 14606.0: 2782}


looking at both the tabs for year 2009 and 2010 of the dataset we have most entries of the user with ID <b>14911</b>  5710 + 5903 = 11613  
user with ID <b>17841</b> = 13097


In [61]:
df1_sorted = df1.set_index('InvoiceDate')
df1_sorted.sort_index()

,Invoice,StockCode,Description,Quantity,Price,Customer ID,Country
InvoiceDate,,,,,,,
2009-12-01 07:45:00,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,6.95,13085.0,United Kingdom
2009-12-01 07:45:00,489434,79323P,PINK CHERRY LIGHTS,12,6.75,13085.0,United Kingdom
2009-12-01 07:45:00,489434,79323W,WHITE CHERRY LIGHTS,12,6.75,13085.0,United Kingdom
2009-12-01 07:45:00,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2.10,13085.0,United Kingdom
2009-12-01 07:45:00,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,1.25,13085.0,United Kingdom
...,...,...,...,...,...,...,...
2010-12-09 20:01:00,538171,22271,FELTCRAFT DOLL ROSIE,2,2.95,17530.0,United Kingdom
2010-12-09 20:01:00,538171,22750,FELTCRAFT PRINCESS LOLA DOLL,1,3.75,17530.0,United Kingdom
2010-12-09 20:01:00,538171,22751,FELTCRAFT PRINCESS OLIVIA DOLL,1,3.75,17530.0,United Kingdom


In [ ]:
df2_sorted = df2.set_index('InvoiceDate')
df2_sorted.sort_index()

,Invoice,StockCode,Description,Quantity,Price,Customer ID,Country
InvoiceDate,,,,,,,
2010-12-01 08:26:00,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2.55,17850.0,United Kingdom
2010-12-01 08:26:00,536365,71053,WHITE METAL LANTERN,6,3.39,17850.0,United Kingdom
2010-12-01 08:26:00,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2.75,17850.0,United Kingdom
2010-12-01 08:26:00,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,3.39,17850.0,United Kingdom
2010-12-01 08:26:00,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,3.39,17850.0,United Kingdom
...,...,...,...,...,...,...,...
2011-12-09 12:50:00,581587,22899,CHILDREN'S APRON DOLLY GIRL,6,2.10,12680.0,France
2011-12-09 12:50:00,581587,23254,CHILDRENS CUTLERY DOLLY GIRL,4,4.15,12680.0,France
2011-12-09 12:50:00,581587,23255,CHILDRENS CUTLERY CIRCUS PARADE,4,4.15,12680.0,France
